In [37]:
import copy

#check if board is solved
def is_solved(board):
    for x in range(len(board)):
        for y in range(len(board)):
            if board[x][y] == 0:
                return False
    return True

#check if there is only one option within the possible_matrix
def easy_move(board, possible_matrix):
    for x in range(len(board)):
        for y in range(len(board)):
            if board[x][y] == 0 and len(possible_matrix[x][y]) == 1:
                return True, x, y, possible_matrix[x][y][0]
    return False, 0, 0, 0

#update possible matrix
def update_possible_matrix(vert, hori, number, possible_matrix):
    for i in range(len(possible_matrix)):
        if number in possible_matrix[i][hori]:
            possible_matrix[i][hori].remove(number)
        if number in possible_matrix[vert][i]:
            possible_matrix[vert][i].remove(number)

#     if hori == vert:
#         for i in range(len(possible_matrix)):
#             if number in possible_matrix[i][i]:
#                 possible_matrix[i][i].remove(number)
#     if hori + vert == len(possible_matrix) - 1:
#         for i in range(len(possible_matrix)):
#             if number in possible_matrix[i][len(possible_matrix) - 1 - i]:
#                 possible_matrix[i][len(possible_matrix) - 1 - i].remove(number)
    
    #only works with 9x9
    base_vertical = vert // 3 * 3
    base_horizontal = hori // 3 * 3
    
    for v in range(base_vertical, base_vertical + 3):
        for h in range(base_horizontal, base_horizontal + 3):
            if number in possible_matrix[v][h]:
                possible_matrix[v][h].remove(number)
                
    possible_matrix[vert][hori] = [number]
    
    return possible_matrix

def initialize_possible_matrix(board):
    digits = list(range(1, len(board) + 1))
    possible_matrix = [[[x for x in digits]for x in digits]for x in digits]
    
    for x in range(len(board)):
        for y in range(len(board)):
            if board[x][y] != 0:
                possible_matrix = update_possible_matrix(x, y, board[x][y], possible_matrix)
    
    return possible_matrix

#print board
def print_board(board):
    for row in board:
        print(row)

def print_possible_matrix(possible_matrix):
    print("This is the possible matrix")
    for x in range(len(possible_matrix)):
        for y in range(len(possible_matrix)):
            print("{} {}".format(x, y), end = " ")
            print(possible_matrix[x][y])

# Takes a 81-digit string and converts it into a 9x9 array
def stringToBoard(s):
    if not(s.isnumeric() and len(s) == 81):
      print("Invalid string! Must be an 81-digit string of numbers")
      return
    board = []
    for i in range(9):
        row = []
        for j in range(9):
            row.append(int(s[9*i + j]))
        board.append(row)
    return board

def dead_end(possible_matrix):
    for x in range(len(possible_matrix)):
        for y in range(len(possible_matrix)):
            if not possible_matrix[x][y]:
                return True
    return False

def find_least_guess(possible_matrix):
    guess_size = len(possible_matrix)
    vert = 0
    hori = 0
    for x in range(len(possible_matrix)):
        for y in range(len(possible_matrix)):
            if len(possible_matrix[x][y]) > 1 and len(possible_matrix[x][y]) < guess_size:
                vert = x
                hori = y
                guess_size = len(possible_matrix[x][y])
    return vert, hori

def sudoku_solver(board, possible_matrix, moves = 0):
    if is_solved(board):
        print("FINISHED BOARD")
        print_board(board)
        print("MOVES: {}".format(moves))
        return True
    elif dead_end(possible_matrix):
        return False
    else:
        while True:
            # Keep doing easy guesses as long as we can
            easy_possible, vert, hori, num = easy_move(board, possible_matrix)
            if easy_possible:
                board[vert][hori] = num
                possible_matrix = update_possible_matrix(vert, hori, num, possible_matrix)
                
                moves += 1
                
                if is_solved(board):
                    print("FINISHED BOARD")
                    print_board(board)
                    print("MOVES: {}".format(moves))
                    return True
            #If we can't, do a recursive call
            else:
                vert, hori = find_least_guess(possible_matrix)
                for guess in possible_matrix[vert][hori]:
                    guess_board = copy.deepcopy(board)
                    guess_board[vert][hori] = guess
                    guess_possible_matrix = copy.deepcopy(possible_matrix)
                    
                    moves += 1
                    
                    guess_possible_matrix = update_possible_matrix(vert, hori, guess, guess_possible_matrix)
                    possible = sudoku_solver(guess_board, guess_possible_matrix, moves)
                    if possible:
                        return True
                return False

def create_boards(input_string):
    return stringToBoard(input_string), stringToBoard(input_string)
#main
# List of board strings:
#http://forum.enjoysudoku.com/patterns-game-results-t6291.html
# empty board = "000000000000000000000000000000000000000000000000000000000000000000000000000000000"

input_string = "000004000800000007007053060005400000000800020900025300010000900000200000008067030"

original_board, board = create_boards(input_string)

print("ORIGINAL BOARD")
print_board(original_board)

possible_matrix = initialize_possible_matrix(board)
solved = sudoku_solver(board, possible_matrix)
if solved:
    print("SOLVED!!!")
else:
    print("IMPOSSIBLE!!!")

ORIGINAL BOARD
[0, 0, 0, 0, 0, 4, 0, 0, 0]
[8, 0, 0, 0, 0, 0, 0, 0, 7]
[0, 0, 7, 0, 5, 3, 0, 6, 0]
[0, 0, 5, 4, 0, 0, 0, 0, 0]
[0, 0, 0, 8, 0, 0, 0, 2, 0]
[9, 0, 0, 0, 2, 5, 3, 0, 0]
[0, 1, 0, 0, 0, 0, 9, 0, 0]
[0, 0, 0, 2, 0, 0, 0, 0, 0]
[0, 0, 8, 0, 6, 7, 0, 3, 0]
FINISHED BOARD
[5, 2, 6, 7, 8, 4, 1, 9, 3]
[8, 3, 9, 6, 1, 2, 4, 5, 7]
[1, 4, 7, 9, 5, 3, 8, 6, 2]
[2, 8, 5, 4, 3, 6, 7, 1, 9]
[3, 6, 1, 8, 7, 9, 5, 2, 4]
[9, 7, 4, 1, 2, 5, 3, 8, 6]
[6, 1, 2, 3, 4, 8, 9, 7, 5]
[7, 5, 3, 2, 9, 1, 6, 4, 8]
[4, 9, 8, 5, 6, 7, 2, 3, 1]
MOVES: 61
SOLVED!!!
